In [3]:
import boto3

# Use a specific AWS profile
session = boto3.Session(profile_name="ajw5296-eas")  # Replace with your AWS profile
rds_client = session.client('rds', region_name='us-east-1')  # Replace with your desired region

# Define your IP address
my_ip_address = "174.59.186.6/32"  # Replace with your public IP address and add /32 for single IP access

def create_security_group():
    ec2_client = session.client('ec2',region_name='us-east-1')
    
    # Create a security group
    response = ec2_client.create_security_group(
        GroupName='RDSRestrictedAccessSG',
        Description='Security group for RDS instance with restricted access'
    )
    security_group_id = response['GroupId']
    
    # Allow inbound access from your IP
    ec2_client.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpProtocol='tcp',
        FromPort=5432,  # Database port, replace if using MySQL (3306)
        ToPort=5432,
        CidrIp=my_ip_address
    )
    
    # Allow inbound access from the VPC (for Lambda and EC2 instances)
    ec2_client.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpProtocol='tcp',
        FromPort=5432,
        ToPort=5432,
        CidrIp='10.0.0.0/16'  # Adjust to your VPC CIDR range
    )
    
    print("Security group created and configured.")
    return security_group_id

def create_rds_instance(security_group_id):
    try:
        response = rds_client.create_db_instance(
            DBName='eas500db',
            DBInstanceIdentifier='pollutionproject2',
            AllocatedStorage=20,
            DBInstanceClass='db.t3.micro',  # Use an appropriate instance type
            Engine='postgres',  # Replace with your preferred engine
            MasterUsername='ajw5296',
            MasterUserPassword='Dont#$with%^any1',  # Use a secure password
            VpcSecurityGroupIds=[security_group_id],
            PubliclyAccessible=False,
            BackupRetentionPeriod=7,
            Tags=[{'Key': 'Name', 'Value': 'SecureDatabase'}],
            MultiAZ=False,
            StorageEncrypted=True
        )
        
        print("Creating secure RDS instance...")
        return response
    except Exception as e:
        print("Error creating RDS instance:", e)

# Create the security group and use it to create the RDS instance
sg_id = create_security_group()
create_rds_instance(sg_id)


Security group created and configured.
Creating secure RDS instance...


{'DBInstance': {'DBInstanceIdentifier': 'pollutionproject2',
  'DBInstanceClass': 'db.t3.micro',
  'Engine': 'postgres',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'ajw5296',
  'DBName': 'eas500db',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '03:57-04:27',
  'BackupRetentionPeriod': 7,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-041e444487afe36a0',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'default',
   'DBSubnetGroupDescription': 'default',
   'VpcId': 'vpc-0460211acfebf52ff',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-0a08bc5e6335e9c53',
     'SubnetAvailabilityZone': {'Name': 'us-east-1b'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-0d5c74a7764b47e05',
     'SubnetAvailabilityZone': {'Name': 'us-east-1d'},
     